In [1]:
import torch
import os 
import sklearn 
import pandas as pd 
import numpy as np
from carla.data.catalog import CsvCatalog

/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [2]:
#TODO How to get access to test set from CARLA ? 
name='credit'
dataset = pd.read_csv(f'../data/{name}/{name}.csv')
            #TODO Better way for defining continous varaibles ?
continuous_wachter = dataset.drop(columns=['label']).columns
dataset = CsvCatalog(file_path=f'../data/{name}/{name}.csv',
                     continuous=continuous_wachter,
                     categorical=[],
                     immutables=[],
                     target='label',
                     scaling_method='Identity')
dataset._df_test

,label,x1,x2,x3,x4,x6,x5,x7
3524,1.0,0.0,8.213834,-0.060784,2.023366,0.578445,-2.639457,1.001406
4233,1.0,1.0,-6.506218,-0.143769,1.495111,1.892873,1.456092,4.704880
3669,0.0,0.0,-9.061529,-0.044144,-0.771386,1.058396,2.107391,-3.136893
2493,0.0,0.0,-0.490417,-0.125385,3.350784,-1.068778,5.648590,3.048535
2498,1.0,1.0,-4.619242,0.064078,-3.029356,-0.481238,-0.470182,-0.398781
...,...,...,...,...,...,...,...,...
8739,1.0,1.0,-13.675538,0.119153,-2.637911,3.391341,-5.401432,-0.428763
9796,0.0,1.0,-10.539815,-0.080166,0.481599,-0.739150,-0.722629,-4.426046
2450,1.0,0.0,-8.951653,-0.236009,-5.504142,-2.892297,-10.912490,0.614561
8276,0.0,1.0,6.986906,-0.006216,2.575797,0.665860,0.951608,1.358329


In [3]:
from string import digits
import re
model_name=['0']
accuracy=['0']
precision=['0']
recall=['0']
auc=['0']
df=pd.DataFrame([])

for name in os.listdir('./Model/'):
    #print(name.split('.')[0][-1].isdigit())
    #print(name)
    if name.split('.')[0][-1].isdigit() or name.startswith('Linear')or name.startswith('Forest'):
        full_name=name
        model=torch.load(f'./Model/{name}')
        name=name.split('_')[1]
        name=name.replace('.pth','')
        #print('FIRST',name)
        #print(name[:-1])
        if name[-1].isdigit():
            name=name[:-1]
        #print('Second',name)

        dataset = pd.read_csv(f'../data/{name}/{name}.csv')
        continuous_wachter = dataset.drop(columns=['label']).columns
        scaler='Identity'
        if name == 'economic':
            scaler='MinMax'

        dataset = CsvCatalog(file_path=f'../data/{name}/{name}.csv',
                     continuous=continuous_wachter,
                     categorical=[],
                     immutables=[],
                     target='label',
                     scaling_method=scaler)
        dataset=dataset._df_test
        data_x, data_y=dataset.drop(columns=['label']), dataset['label']
        data_pred= model.predict(data_x)
        model_name.append(full_name)
        print(data_pred.shape)
        print(data_y.shape)
        if len(data_pred.shape)>1:
            data_pred= np.argmax(data_pred, axis=1)

        accuracy.append(sklearn.metrics.accuracy_score(data_y,data_pred))
        precision.append(sklearn.metrics.precision_score(data_y,data_pred))
        recall.append(sklearn.metrics.recall_score(data_y,data_pred))
        auc.append(sklearn.metrics.roc_auc_score(data_y,data_pred))
df['model_name']=model_name
df['accuracy']=accuracy
df['precision']=precision
df['recall']=recall
df['auc'] =auc
df

(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)


/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model

(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)


/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice 

,model_name,accuracy,precision,recall,auc
0,0,0,0,0,0
1,Forest_credit.pth,0.8104,0.823915,0.795257,0.810584
2,Forest_economic.pth,0.8032,0.75778,0.815385,0.804467
3,Forest_sanity-3-lin.pth,0.7972,0.810945,0.778043,0.797308
4,Forest_sanity-3-non-add.pth,0.8588,0.936508,0.469745,0.729531
5,Forest_sanity-3-non-lin.pth,0.968,0.971329,0.959657,0.967469
6,Linear_credit.pth,0.754,0.768473,0.737589,0.754253
7,Linear_economic.pth,0.8152,0.780109,0.79461,0.812684
8,Linear_sanity-3-lin.pth,0.998,0.997685,0.998456,0.997983
9,Linear_sanity-3-non-add.pth,0.9984,0.993779,1,0.998925


In [4]:
# Table to Latex 
# Get header
st=''
col=[]
name=[]
df=df.iloc[1:]
n_saver=[]
key=[]
for index, row in df.iterrows():
    #print(row['model_name'].split('_'))
    #if st is '':
    #    st += row['model_name'].split('_')[1].replace('.pth','')[:-1]
    #else:
    #    st += ' & '+row['model_name'].split('_')[1].replace('.pth','')[:-1]
    if row['model_name'].split('_')[1].replace('.pth','')[-1].isdigit():
        print('is Digit')
        data=row['model_name'].split('_')[0]+ row['model_name'].split('_')[1].replace('.pth','')[-1]
    else: 
        data=row['model_name'].split('_')[0]
    col.append(data)
    name.append(row['model_name'].split('_')[1].replace('.pth','')[:-1])
    key.append(row['model_name'])
st+= ' \\\\ '
col= np.unique(col)
name = np.unique(name[name!='credi'])
name=['economic','credit','sanity-3-lin','sanity-3-non-lin','sanity-3-non-add']
for n in name :
    st += ' & '+ n

print('col',col)
print('name',name)
print('key', key)
for i in range(0,len(col)):
    st += col[i] 
    num=''
    if col[i][-1].isdigit():
        num = col[i][-1]
    for j in range(0,len(name)):
        #print(''col[i])
        #print(name[i])
        for k in key: 
            print(f'{name[j]}{num}')
            if col[i] in k and f'{name[j]}{num}' in k: 
                t=k

        print(t)
        #print(f'{col[i]}_{name[i]}')
        #print(str(df[df['model_name'] ==f'{col[i]}_{name[j]}']['accuracy'].values[0]))
        #print(df[df['model_name'] ==f'{col[i]}_{name[i]}']['accuracy'])
        st += ' & ' + str(df[df['model_name'] ==f'{t}']['accuracy'].values[0])
    st+=' \\\\ '
print(st)

is Digit
is Digit
is Digit
is Digit
is Digit
col ['Forest' 'Linear' 'MLP0']
name ['economic', 'credit', 'sanity-3-lin', 'sanity-3-non-lin', 'sanity-3-non-add']
key ['Forest_credit.pth', 'Forest_economic.pth', 'Forest_sanity-3-lin.pth', 'Forest_sanity-3-non-add.pth', 'Forest_sanity-3-non-lin.pth', 'Linear_credit.pth', 'Linear_economic.pth', 'Linear_sanity-3-lin.pth', 'Linear_sanity-3-non-add.pth', 'Linear_sanity-3-non-lin.pth', 'MLP_credit0.pth', 'MLP_economic0.pth', 'MLP_sanity-3-lin0.pth', 'MLP_sanity-3-non-add0.pth', 'MLP_sanity-3-non-lin0.pth']
economic
economic
economic
economic
economic
economic
economic
economic
economic
economic
economic
economic
economic
economic
economic
Forest_economic.pth
credit
credit
credit
credit
credit
credit
credit
credit
credit
credit
credit
credit
credit
credit
credit
Forest_credit.pth
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin
sanity-3-lin